In [2]:
import pandas as pd
from pandas.io import sql

Pandas can be used to connect to most relational databases. In this demonstration, we will create and connect to a SQLite database. SQLite creates portable SQL databases saved in a single file. These databases are stored in a very efficient manner and allow fast querying, making them ideal for small databases or databases that need to be moved across machines.

If you are looking to start using a database without the setup of `mysql` or `postgres`, SQLite is a good start.

In [3]:
import sqlite3

In [4]:
conn = sqlite3.connect('dat-test.db')

Let's return to the Rossmann sales data and load that into the database.

Data is moved to the database through the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arugments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

In [8]:
data = pd.read_csv('../../assets/dataset/rossmann.csv', low_memory=False)
data.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270,9,2008,0,NaN,NaN,NaN
1,2,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct"
2,3,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct"
3,4,c,c,620,9,2009,0,NaN,NaN,NaN
4,5,a,a,29910,4,2015,0,NaN,NaN,NaN


In [14]:
data.to_sql('rossmann_sales',
            con=conn,
            if_exists='replace',
            index=False)

In [10]:
sql.read_sql('select * from rossmann_sales limit 10', con=conn)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270,9,2008,0,NaN,NaN,None
1,2,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct"
2,3,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct"
3,4,c,c,620,9,2009,0,NaN,NaN,None
4,5,a,a,29910,4,2015,0,NaN,NaN,None
5,6,a,a,310,12,2013,0,NaN,NaN,None
6,7,a,c,24000,4,2013,0,NaN,NaN,None
7,8,a,a,7520,10,2014,0,NaN,NaN,None
8,9,a,c,2030,8,2000,0,NaN,NaN,None
9,10,a,a,3160,9,2009,0,NaN,NaN,None


> #### CHECK: Load the Rossmann Store metadata in `rossmann-stores.csv` and create a table into the database from it

In [15]:
rossmann_stores = pd.read_csv('../../assets/dataset/rossmann.csv')
rossmann_stores.to_sql('rossmann_stores', if_exists='replace', index=False, con=conn)

> #### CHECK: (SELECT) Have the students write a query that returns the `Store`, `Date` and `Customers`

In [16]:
sql.read_sql(
"""
SELECT 
Store, Date, Customers 
FROM rossmann_sales;
""", con=conn).head()

ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



DatabaseError: Execution failed on sql '
SELECT 
Store, Date, Customers 
FROM rossmann_sales;
': no such column: Date

> #### CHECK: (WHERE) Have the students write a query that returns the `Store`, `Date` and `Customers` for when the stores were open and running a promotion

In [75]:
sql.read_sql(
"""
SELECT 
Store, Date, Customers 
FROM rossmann_sales
WHERE Open = 1 and Promo = 1;
""", con=conn).head()

,Store,Date,Customers
0,1,2015-07-31,555
1,2,2015-07-31,625
2,3,2015-07-31,821
3,4,2015-07-31,1498
4,5,2015-07-31,559


> #### CHECK: (GROUP BY) Have the students write a query that returns the total sales on promotion days.

In [78]:
sql.read_sql(
"""
SELECT
Promo, Store, SUM(Sales)
FROM rossmann_sales
GROUP BY Promo
""", con=conn).head()

,Promo,Store,SUM(Sales)
0,0,1115,2771974337
1,1,1115,3101206286


#### Exercises

1. Load the Walmart sales and store features data
1. Create a table for each of those datasets
1. Select the store, date and fuel price on days it was over 90 degrees
1. Select the store, date and weekly sales and temperature
1. What were average sales on holiday vs. non-holiday sales
1. What were average sales on holiday vs. non-holiday sales when the temperature was below 32 degrees

In [17]:
# Load the Walmart sales and store features data
walmart_sales = pd.read_csv('../../assets/dataset/walmart-sales.csv')
walmart_sales.head()

walmart_features = pd.read_csv('../../assets/dataset/features.csv')
walmart_features.head()

IOError: File ../../assets/dataset/walmart-sales.csv does not exist

In [80]:
# Create a table for each of those datasets
walmart_sales.to_sql('walmart_sales', index=False, if_exists='replace', con = conn)
walmart_features.to_sql('walmart_features', if_exists='replace', index=False, con=conn)

In [81]:
# Select the store, date and fuel price on days it was over 90 degrees
query = """
SELECT 
Date, Store, Fuel_Price, Temperature
FROM walmart_features
WHERE Temperature > 90
"""
sql.read_sql(query, con = conn).head()

,Date,Store,Fuel_Price,Temperature
0,2011-08-05,1,3.684,91.65
1,2011-08-12,1,3.638,90.76
2,2011-07-29,2,3.682,90.07
3,2011-08-05,2,3.684,93.34
4,2011-08-12,2,3.638,91.58


In [82]:
# Select the store, date and weekly sales and temperature
query = """
SELECT 
s.Store, s.Date, s.Weekly_Sales, f.Temperature
FROM walmart_sales as s
JOIN walmart_features as f
ON s.Store = f.Store and s.Date = f.Date
"""
sql.read_sql(query, con = conn).head()

,Store,Date,Weekly_Sales,Temperature
0,1,2010-02-05,24924.50,42.31
1,1,2010-02-12,46039.49,38.51
2,1,2010-02-19,41595.55,39.93
3,1,2010-02-26,19403.54,46.63
4,1,2010-03-05,21827.90,46.50


In [83]:
# What were average sales on holiday vs. non-holiday sales
query = """
SELECT 
IsHoliday, AVG(Weekly_Sales)
FROM walmart_sales as s
GROUP BY IsHoliday
"""
sql.read_sql(query, con = conn)

,IsHoliday,AVG(Weekly_Sales)
0,0,15901.445069
1,1,17035.823187


In [60]:
# What were average sales on holiday vs. non-holiday sales when the temperature was below 32 degrees
query = """
SELECT 
s.IsHoliday, AVG(s.Weekly_Sales)
FROM walmart_sales as s
JOIN walmart_features as f
ON s.Store = f.Store and s.Date = f.Date
WHERE f.Temperature < 32
GROUP BY s.IsHoliday
"""
sql.read_sql(query, con = conn)

,IsHoliday,AVG(s.Weekly_Sales)
0,0,15275.770307
1,1,15111.471238
